# I. Preliminaries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import math
from xgboost import XGBClassifier as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve,confusion_matrix,auc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from xgboost import plot_importance
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
from matplotlib import pyplot
import time

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
#download dataset from kaggle and adjust path below
train_data = pd.read_csv('../input/csci-e-82-2021/train_data.csv')
test_data = pd.read_csv('../input/csci-e-82-2021/test_data.csv')

### Data Preparation

In [5]:
# train_data, test_data -> train, test
# - drop output column
# - convert subject, phase, and state to numerical values (one-hot encoding)
# - drop cols with only one value

train = train_data.drop(columns=['output'],axis=1) # drop output col
subject_cols = pd.get_dummies(train['subject'], prefix='subject') # one hot encode cols with categorical values
phase_cols = pd.get_dummies(train['phase'], prefix='phase')
state_cols = pd.get_dummies(train['state'], prefix='state')
train = pd.concat([train,subject_cols, phase_cols, state_cols],axis=1) # add one hot encoding
train.drop(['subject', 'phase', 'state'],axis=1, inplace=True) # remove original cols now that we have one hot encoding

subject_cols_test = pd.get_dummies(test_data['subject'], prefix='subject') # one hot encode cols with categorical values
phase_cols_test = pd.get_dummies(test_data['phase'], prefix='phase')
state_cols_test = pd.get_dummies(test_data['state'], prefix='state')
test = pd.concat([test_data,subject_cols_test, phase_cols_test, state_cols_test],axis=1) # do the same for test
test.drop(['subject', 'phase', 'state'],axis=1, inplace=True) 

one_value_col=[]
for col in train.columns:
    if train[col].nunique() == 1:
        #print(train_data[col].nunique())
        one_value_col.append(col)
train = train.drop(columns=one_value_col, axis=1)
test = test.drop(columns=one_value_col, axis=1)

# drop column which are not important at all
#drop_feat = ['y19','x32','x29','x27','x23','x19','x16','x13','y167','z27','x133','z25','z23','y171','x80','y175','z19','z16','x158','z183','z180','x196','y13','y23','y25','z106','y27','z80','x209','x184','x172','x183','x180','z162','x174','z171','y80','z174','y16','z13']
#train = train.drop(columns=drop_feat,axis=1)

### Split train data set

In [6]:
X = train
y = train_data['output']

# get subset of cols that are not the one hot encodings
numeric_feat = list(set(train.columns) - set(train.iloc[:, 558:].columns))

# standardize X
scale = StandardScaler()
X[numeric_feat] = scale.fit_transform(X[numeric_feat])

# split 
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,stratify=y)
X_test = test

# standard splits 
X_train[numeric_feat] = scale.fit_transform(X_train[numeric_feat])
X_val[numeric_feat] = scale.transform(X_val[numeric_feat])
X_test[numeric_feat] = scale.transform(test[numeric_feat])

### Define class weight

In [7]:
classes_weights = class_weight.compute_class_weight('balanced',
                                             np.unique(y_train),
                                             y_train)
weight = {0: 0.80 , 1: 0.20}
sample_weight=compute_sample_weight("balanced", y_train)

#sample_weight=compute_sample_weight("balanced", y)
#sample_weight = y_train.apply(lambda x : weight[x])

In [8]:
y_train.value_counts()
scale_pos_weight=533/2905
scale_pos_weight

# II. Approach

## We will be working exclusively with XGBoost. (We tested RFC and it gave poorer performance than XGBoost despite tuning).

1. Plot baseline XGBoost model
2. Assess baseline performance and understand feature importance
3. Test core hyperparamters (# of trees and depth of trees) using accuracy and f1 score
4. Optimize model
5. Retest feature importance for optimized model
6. Test optimal threshold (to help with imbalaned data, as is here)


# III. XGBoost Tuning

## XGBoost base model

In [9]:
model = xgb(random_state=42)
preds = model.fit(X_train, y_train).predict(X_val)
print('Train accuracy %s:' % model.score(X_train, y_train)) # training accuracy = 1.0
print('Validation accuracy %s:' % accuracy_score(preds, y_val)) # validation accuracy = 0.864

In [10]:
# plots of feature importance for base model
fig, ax = plt.subplots(figsize=(24, 10))
plot_importance(model, ax=ax)

plt.figure(figsize=(24, 10))
plt.plot(range(len(model.feature_importances_)), model.feature_importances_)
plt.show()

In [11]:
# observe matrix (easier to view than cluttered plots)
base_featimport = model.feature_importances_.reshape((1, 578)) # we want (578, 1)
base_featimport = pd.DataFrame(base_featimport, columns=X_train.columns).T
base_featimport = base_featimport.sort_values(by=0, ascending=False)

# look at top 25 features
base_featimport[:25]

In [12]:
# replot top 25 features
# we observe there are 5 features that contribute most significantly 
plt.figure(figsize=(24, 10))
plt.plot(range(len(base_featimport[:25])), base_featimport[:25].to_numpy())
plt.show()

In [ ]:
# baseline with CV (using original non-split data)
# purpose: to reduce variance inherent to simpler test/train split approach
# using stratified CV because classes are imbalanced
model = xgb()
kfold = StratifiedKFold(n_splits=5, random_state=42)
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)) # 0.863 accuracy close to test/train split run

In [ ]:
# initial run of hyperparameter tuning
# testing 3 hyperparameters
# - n_estimators
# - max_depth
# - scale_pos_weight
def xgb_param_selection(X, y):
    n_estimators = [25, 50, 75] # of trees (too large results in overfitting)
    max_depth=[6, 8] # depth of each tree (too large results in overfitting)
    scale_pos_weight = [0.18, 0.33] 
    learning_rate = [0.1]
    objective = ['reg:logistic']
    param_grid = {'n_estimators': n_estimators, 
                  'max_depth' : max_depth,
                  'scale_pos_weight':scale_pos_weight, 
                  'learning_rate': learning_rate,
                  'objective':objective}
    
    # build model
    boost = xgb(use_label_encoder=False)
    grid_search = RandomizedSearchCV(boost, param_grid, cv=3, n_jobs=-1,verbose=0)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search,grid_search.best_params_

start_time = time.time()
xgb_model,xgb_best_prama = xgb_param_selection(X_train,y_train)
execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)
print('Best param of XGB : ',xgb_best_prama)

In [ ]:
# Best param from previous run:  
# {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 75, 'objective': 'reg:logistic', 'scale_pos_weight': 0.33}
# Because the best values for max_depth and n_estimators were the highest in the list,
# we will test run again with higher values

def xgb_param_selection(X, y):
    n_estimators = [75, 100, 150] # of trees (too large results in overfitting)
    max_depth=[8, 15, 25] # depth of each tree (too large results in overfitting)
    scale_pos_weight = [0.33, 0.37] 
    learning_rate = [0.1]
    objective = ['reg:logistic']
    param_grid = {'n_estimators': n_estimators, 
                  'max_depth' : max_depth,
                  'scale_pos_weight':scale_pos_weight, 
                  'learning_rate': learning_rate,
                  'objective':objective}
    
    # build model
    boost = xgb(use_label_encoder=False)
    grid_search = RandomizedSearchCV(boost, param_grid, cv=3, n_jobs=-1,verbose=0)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search,grid_search.best_params_

start_time = time.time()
xgb_model,xgb_best_prama = xgb_param_selection(X_train,y_train)
execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)
print('Best param of XGB : ',xgb_best_prama)

In [ ]:
# best parameters continue to be:
# {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 75, 'objective': 'reg:logistic', 'scale_pos_weight': 0.33}
# this was unexpected as we thought that increasing the estimators and depth would improve accuracy
# this yields ~10% improvement in validation accuracy
model = xgb(eta=0.1, max_depth=8, n_estimators=75, objective='reg:logistic', scale_pos_weight=0.33) 
preds = model.fit(X_train, y_train).predict(X_val)
print('Train accuracy %s:' % model.score(X_train, y_train)) # training accuracy continues to be 1.00
print('Validation accuracy %s:' % accuracy_score(preds, y_val)) # validation accuracy 0.8577

In [ ]:
# testing next:
# - colsample_bytree [0.5, 0.55, 0.6, 0.65, 0.7, 0.8] -> expect higher colsample to yield better results because...
# - n_estimators [75, 80, 90] -> re-testing because we change other parameters and this is co-dependent
# - eval_metric ['error', 'logloss'] -> expect logloss to yield better results as its more appropriate
# - min_child_weight [0.5, 1, 3, 5]
# best is (eta=0.1, max_depth=8, n_estimators=75, objective='binary:logistic', scale_pos_weight=0.33, colsample_bytree=0.7, min_child_weight=1) 

def xgb_param_selection(X, y):
    n_estimators = [75, 80, 90] # of trees (too large results in overfitting)
    max_depth=[8] # depth of each tree (too large results in overfitting)
    scale_pos_weight = [0.33] 
    learning_rate = [0.1]
    objective = ['reg:logistic']
    eval_metric = ['error', 'logloss']
    min_child_weight = [0.5, 1, 3, 5]
    colsample_bytree = [0.5, 0.6, 0.7, 0.8]
    param_grid = {'n_estimators': n_estimators, 
                  'max_depth' : max_depth,
                  'scale_pos_weight':scale_pos_weight, 
                  'learning_rate': learning_rate,
                  'objective':objective, 
                  'eval_metric':eval_metric,
                  'colsample_bytree':colsample_bytree,
                  'min_child_weight': min_child_weight}
    
    # build model
    boost = xgb(use_label_encoder=False)
    grid_search = RandomizedSearchCV(boost, param_grid, cv=3, n_jobs=-1,verbose=0)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search,grid_search.best_params_

start_time = time.time()
xgb_model,xgb_best_prama = xgb_param_selection(X, y)
execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)
print('Best param of XGB : ',xgb_best_prama)

In [ ]:
# best parameters after 2 grid search runs:
# {'scale_pos_weight': 0.33, 'objective': 'reg:logistic', 'n_estimators': 90, 'min_child_weight': 3, 'max_depth': 8, 'learning_rate': 0.1, 'eval_metric': 'error', 'colsample_bytree': 0.6}
model = xgb(eta=0.1, max_depth=8, n_estimators=90, objective='reg:logistic', scale_pos_weight=0.33, min_child_weight=3, colsample_bytree=0.6, random_state=42, eval_metric='error') 
preds = model.fit(X_train, y_train).predict(X_val) 
print('Train accuracy: %s' % model.score(X_train, y_train)) 
print('Validation accuracy: %s' % accuracy_score(preds, y_val)) # validation accuracy 0.866

In [ ]:
# investigating effect of number of trees (n_estimators) on tuned model
acc_scores = []
f1_scores = []
start_time = time.time()
trees = [50, 70, 90, 110, 130, 150]
for ne in trees:
    model = xgb(eta=0.1, max_depth=8, n_estimators=ne, objective='reg:logistic', scale_pos_weight=0.33, min_child_weight=3, colsample_bytree=0.6, eval_metric='logloss') 
    acc_score_list = cross_val_score(model, X, y, cv=3,n_jobs=-1) #returns accuracy
    f1_score_list = cross_val_score(model, X, y, cv=3, scoring='f1')
    acc_scores.append(acc_score_list)
    f1_scores.append(f1_score_list)
execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)

In [ ]:
# plot accuracy for # of trees
box_df = pd.DataFrame([np.arange(10,150,10), acc_scores]).T
box_df.columns = ['numoftrees','acc_score']
box_df = pd.DataFrame(np.array(acc_scores))
box_df['numoftrees'] = np.arange(10,150,10)
box_df = box_df.melt(id_vars='numoftrees')

plt.figure(figsize=(12,5))
sns.boxplot(x="numoftrees", 
            y="value", 
            data=box_df, 
            showmeans=True);
plt.xlabel('Number of trees')
plt.ylabel('Classification score')
plt.title('Classification score as a function of the number of trees');

In [ ]:
# plot f1 score for # of trees
box_df = pd.DataFrame([np.arange(10,150,10), f1_scores]).T
box_df.columns = ['numoftrees','f1_score']
box_df = pd.DataFrame(np.array(f1_scores))
box_df['numoftrees'] = np.arange(10,150,10)
box_df = box_df.melt(id_vars='numoftrees')

plt.figure(figsize=(12,5))
sns.boxplot(x="numoftrees", 
            y="value", 
            data=box_df, 
            showmeans=True);
plt.xlabel('Number of trees')
plt.ylabel('F1 score')
plt.title('F1 score as a function of the number of trees');

In [ ]:
# find optimal threshold
yhat = model.predict_proba(X_val)
yhat = yhat[:, 1] # keep probabilities for the positive outcome only
fpr, tpr, thresholds = metrics.roc_curve(y_val, yhat) # y_true score, y score

# using Youden's J statistic used to identify best threshold
J = tpr - fpr 
ix = np.argmax(J)
opt_thresh = thresholds[ix]
print('Optimal predict prob threshold: %f' % (opt_thresh))

In [ ]:
# generate report for optimal threshold (on X_val)
def thresh_pred(model, X, thresh):
    return (model.predict_proba(X)[:,1] > thresh).astype(int)

pred = thresh_pred(model, X_val, opt_thresh)
print(metrics.classification_report(y_val, pred))

In [ ]:
# generate report for optimal threshold (on X)
pred = thresh_pred(model, X, opt_thresh)
print(metrics.classification_report(y, pred))

In [ ]:
# now we figure out how to generate the submission
# we submit y_pred (classification guesses based on our X_test)
# so we need to fit X on our best model
# and then get predictions using thresh_pred -> which we submit

# ....model code below....
test_model = xgb(...)

In [ ]:
submission_df = pd.DataFrame()
y_prob = thresh_pred(test_model, X_test, opt_thresh)

submission_df['id'] = range(0,len(X_test))
submission_df['output'] = np.where(y_prob>opt_thresh,1,0)


submission_df.to_csv('./xg_submission_scale_pos_finetune.csv',index=False)